Prueba de Modelamiento 
===

Por: Alexander Castaño Tamayo

**Contexto:** Dado el riesgo de que se pueda presentar un evento desafortunado, como lo es un incidente de tránsito, una muerte o enfermedad inesperada, etc. Las personas buscan mitigar estos riesgos asegurándose mediante una póliza de diferente tipo, el cual dependerá del motivo o suceso que se desee mitigar (póliza de Salud, Vida o Automóviles). Es por esto, que para la compañía es de vital importancia conocer cuáles de los clientes que tienen una solución actualmente en algunas de sus pólizas de Salud, Vida o Automóviles pueden cancelarla para todo el periodo de tiempo comprendido en el año 2017, con la finalidad de anticiparse al evento en que se cancela alguna solución y realizar campañas de conservación de clientes. 

Se abordará la aplicación empírica de un modelo predictivo como lo es $GLM$ para la identificación de variables o factores que indiquen el posible abandono de una póliza una entidad aseguradora.

Para ello, se utilizará la base de datos con características socio demográficas de los **Clientes**. Esta contiene:

* Fecha_Nacimiento: Fecha de nacimiento del cliente.
* ipo_Identificacion_Cd: Tipo de documento que utiliza para identificarse.
* Sexo_Cd: F es Femenino, M es Masculino.
* Nm_Estrato_Vivienda: Estrato al que pertenece su lugar de vivienda.
* Tipo_Persona_Cd: Natural, Jurídica.
* Numero_Hijos: Cantidad de hijos. 
* Rango_Ingresos_Desc: Rango en el que se encuentran sus ingresos totales por mes.
* Rango_Egresos_Desc: Rango en el que se encuentran todas las salidas de dinero por mes.
* Estado_Civil_cd: Es Casado, Viudo, Divorciado, Soltero, Unión libre.
* Ocupacion_Desc: Los más comunes son empleado/asalariado, jubilado, independiente.
* Profesion_Desc: Profesión a la que se dedica.
* Nivel_Estudios_Desc: Máximo nivel de estudios alcanzado.
* Fecha_Inicio_Compania: Fecha en que inicio vinculación con la compañía.
* Nombre_Departamento: Departamento de contacto.

In [8]:
clientes <- read.csv2('CLIENTES.txt', header = T, na.strings = c('?', '-1'))
head(clientes)

Cliente_Id,Fecha_Nacimiento,Tipo_Identificacion_Cd,Sexo_Cd,Nm_Estrato_Vivienda,Tipo_Persona_Cd,Numero_Hijos,Rango_Ingresos_Desc,Rango_Egresos_Desc,Estado_Civil_cd,Ocupacion_Desc,Profesion_Desc,Nivel_Estudios_Desc,Fecha_Inicio_Compania,Nombre_Departamento
<int>,<fct>,<fct>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,1992/11/09,C,F,NA,N,NA,1 - 1'200.000,1 - 1'200.000,C,OTRA,Sin Informacion,TECNICO,2015/11/12,ANTIOQUIA
2,1983/08/25,C,F,3,N,NA,3'000.001 - 5'000.000,1 - 1'200.000,S,EMPLEADO/ASALARIADO,NEGOCIOS INTERNACIONALES,Sin Informacion,2012/10/29,BOGOTA D.C.
3,1993/03/10,C,F,NA,N,NA,Sin Informacion,Sin Informacion,NA,Sin Informacion,Sin Informacion,Sin Informacion,2015/12/07,VALLE DEL CAUCA
4,1989/12/28,C,M,NA,N,0,Sin Informacion,Sin Informacion,S,ESTUDIANTE,Sin Informacion,Sin Informacion,2015/06/24,BOGOTA D.C.
5,1981/01/30,C,F,2,N,2,1 - 1'200.000,1 - 1'200.000,S,INDEPENDIENTE,Sin Informacion,Sin Informacion,2006/07/14,VALLE DEL CAUCA
6,1995/03/29,C,M,NA,N,NA,Sin Informacion,Sin Informacion,NA,Sin Informacion,Sin Informacion,Sin Informacion,2015/11/17,CALDAS


También, se tienen bases de datos de las pólizas que comenzaron su vigencia entre los años 2014 y 2016, teniendo en cuenta su estado al corte del 31 de diciembre del 2016. Estas bases de datos contiene información de: 

* Poliza_Id: Identificador único para la póliza.
* Fecha_Inicio_Primera_Vigencia: Fecha en que inicia la vigencia de la póliza.
* Fecha_Cancelacion: Fecha de cancelación de la póliza.
* Cliente_Id: Identificador único del Cliente en la compañía.
* Ramos_Id: Hace referencia al tipo de evento que se desea cubrir, el código 26 corresponde a la solución A, el 78 a la solución B y el 16872 a la solución C.

Para este ejercicio, vamos a construir un modelo de deserción para calcular la probabilidad de que los clientes **Naturales** que tenga la solución **C** cancelen su póliza.

In [13]:
# Cargamos la base de datos de pólizas, filtramos las pólizas tipo C y agrupamos en una sóla base de datos con información de clientes
poli2014 <- read.table('POLI2014.txt', header = T, na.strings='?') 
poli2014$año <- 2014
poli2015 <- read.table('POLI2015.txt', header = T, na.strings='?')
poli2015$año <- 2015
poli2016 <- read.table('POLI2016.txt', header = T, na.strings='?')
poli2016$año <- 2016 

poli_c <- rbind.data.frame(
  subset(poli2014, poli2014$Ramos_Id==16872),
  subset(poli2015, poli2015$Ramos_Id==16872),
  subset(poli2016, poli2016$Ramos_Id==16872)
                            )
poli_client <- merge(poli_c, clientes, 
                     by= "Cliente_Id")
head(poli_client)

Cliente_Id,Poliza_Id,Fecha_Inicio_Primera_Vigencia,Fecha_Cancelacion,Ramos_Id,año,Fecha_Nacimiento,Tipo_Identificacion_Cd,Sexo_Cd,Nm_Estrato_Vivienda,Tipo_Persona_Cd,Numero_Hijos,Rango_Ingresos_Desc,Rango_Egresos_Desc,Estado_Civil_cd,Ocupacion_Desc,Profesion_Desc,Nivel_Estudios_Desc,Fecha_Inicio_Compania,Nombre_Departamento
<int>,<int>,<fct>,<fct>,<int>,<dbl>,<fct>,<fct>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,262093074,2015/11/12,NA,16872,2015,1992/11/09,C,F,NA,N,NA,1 - 1'200.000,1 - 1'200.000,C,OTRA,Sin Informacion,TECNICO,2015/11/12,ANTIOQUIA
2,251769194,2015/05/19,2016/05/19,16872,2015,1983/08/25,C,F,3,N,NA,3'000.001 - 5'000.000,1 - 1'200.000,S,EMPLEADO/ASALARIADO,NEGOCIOS INTERNACIONALES,Sin Informacion,2012/10/29,BOGOTA D.C.
5,261757926,2015/11/14,2016/08/14,16872,2015,1981/01/30,C,F,2,N,2,1 - 1'200.000,1 - 1'200.000,S,INDEPENDIENTE,Sin Informacion,Sin Informacion,2006/07/14,VALLE DEL CAUCA
8,255314284,2015/07/22,NA,16872,2015,1990/02/07,C,M,3,N,0,MAS DE 5'000.000,1'200.001 - 3'000.000,S,EMPLEADO/ASALARIADO,INGENIERO,TECNICO,2012/01/02,ANTIOQUIA
22,271897486,2016/04/07,2016/11/29,16872,2016,1990/12/10,C,F,NA,N,NA,1'200.001 - 3'000.000,1 - 1'200.000,S,ABARROTEROS,ENFERMERO,Sin Informacion,2013/02/22,VALLE DEL CAUCA
43,267056618,2016/02/19,NA,16872,2016,1972/02/26,C,F,NA,N,0,1'200.001 - 3'000.000,1'200.001 - 3'000.000,S,EMPLEADO (TIEMPO COMPLETO),CONTADOR,Sin Informacion,2009/11/27,MAGDALENA


---
## Regresión logística
---

La Regresión Logística Simple, desarrollada por David Cox en 1958, es un método de regresión que permite estimar la probabilidad de una variable cualitativa binaria en función de una variable cuantitativa. Una de las principales aplicaciones de la regresión logística es la de clasificación binaria, en el que las observaciones se clasifican en un grupo u otro dependiendo del valor que tome la variable empleada como predictor. Por ejemplo, clasificar a un individuo desconocido como hombre o mujer en función del tamaño de la mandíbula.

Es importante tener en cuenta que, aunque la regresión logística permite clasificar, se trata de un modelo de regresión que modela el logaritmo de la probabilidad de pertenecer a cada grupo. La asignación final se hace en función de las probabilidades predichas.

### ¿Por qué regresión logística y no lineal? ###

Si una variable cualitativa con dos niveles se codifica como 1 y 0, matemáticamente es posible ajustar un modelo de regresión lineal por mínimos cuadrados $β_0$+$β_1$$X$. El problema de esta aproximación es que, al tratarse de una recta, para valores extremos del predictor, se obtienen valores de $Y$ menores que 0 o mayores que 1, lo que entra en contradicción con el hecho de que las probabilidades siempre están dentro del rango [0,1].

Para evitar estos problemas, la regresión logística transforma el valor devuelto por la regresión lineal ($β_0$$+$$β_1$$X$) empleando una función cuyo resultado está siempre comprendido entre 0 y 1. Existen varias funciones que cumplen esta descripción, una de las más utilizadas es la función logística.

En la regresión lineal simple, se modela el valor de la variable dependiente $Y$ en función del valor de la variable independiente $X$. Sin embargo, en la regresión logística, se modela la probabilidad de que la variable respuesta $Y$ pertenezca al nivel de referencia 1 en función del valor que adquieran los predictores, mediante el uso de $LOG$ of $ODDs$.

Supóngase que la probabilidad de que un evento sea verdadero es de 0.8, por lo que la probabilidad de evento falso es de 1 - 0.8 = 0.2. Los $ODDs$ o razón de probabilidad de verdadero se definen como el ratio entre la probabilidad de evento verdadero y la probabilidad de evento falso p/q. En este caso los $ODDs$ de verdadero son 0.8 / 0.2 = 4, lo que equivale a decir que se esperan 4 eventos verdaderos por cada evento falso.

---
### Preparación de la base de datos 
---

En la base de datos consolidad hay algunas variables de fecha, las cuales deben ser modificadas para crear indicadores que den cuenta, por ejemplo, de la edad de los clientes, la antiguedad de la póliza, entre otras. 

In [18]:
# Orgnizamos la variables de fechas y creamos indicadores adicionales
poli_client$Fecha_Inicio_Primera_Vigencia <- as.Date(poli_client$Fecha_Inicio_Primera_Vigencia,'%Y/%m/%d')
poli_client$Fecha_Cancelacion <- as.Date(poli_client$Fecha_Cancelacion,'%Y/%m/%d')
poli_client$Fecha_Nacimiento <- as.Date(poli_client$Fecha_Nacimiento,'%Y/%m/%d')
poli_client$Fecha_Inicio_Compania <- as.Date(poli_client$Fecha_Inicio_Compania, '%Y/%m/%d')
poli_client$antiguedad <- ifelse(is.na(poli_client$Fecha_Cancelacion),
                                 as.Date('2019-06-21')-poli_client$Fecha_Inicio_Compania, 
                                 poli_client$Fecha_Cancelacion-poli_client$Fecha_Inicio_Compania)/365
poli_client$edad <- as.numeric(as.Date('2019-06-21')-poli_client$Fecha_Nacimiento)/365

Estas variables de fecha también son útiles al momento de señalar cuál póliza continua vigente y cuál ha sido cancelada

In [19]:
# Definimos el indicador de cancelación
poli_client$cancelada <- ifelse(is.na(poli_client$Fecha_Cancelacion),0,1)

Ahora bien, las demás variables con información socio demográfica de los clientes deben ser agrupadas y recategorizadas para facilitar su manipulación, visualización e interpretación: 

In [23]:
# generando categorias de la variable ingreso
ingresos.t <- as.numeric(poli_client$Rango_Ingresos_Desc)
ingresos2 <- vector()
ingresos2[ingresos.t==1] <- 2
ingresos2[ingresos.t==2] <- 1
ingresos2[ingresos.t==3] <- 4
ingresos2[ingresos.t==9] <- 3
ingresos2[ingresos.t==19] <- 0 # sin información
ingresos2[is.na(ingresos2)] <- 4
ingresos2 <- as.factor(ingresos2)
poli_client$ingresos <- ingresos2

In [24]:
# generando categorias de la variable egreso
egresos.t <- as.numeric(poli_client$Rango_Egresos_Desc)
egresos2 <- vector()
egresos2[egresos.t==1] <- 2
egresos2[egresos.t==2] <- 1
egresos2[egresos.t==3] <- 4
egresos2[egresos.t==10] <- 3
egresos2[egresos.t==20] <- 0 # sin informaci?n
egresos2[is.na(egresos2)] <- 4
egresos2 <- as.factor(egresos2)
poli_client$egresos <- egresos2

In [25]:
# categorizando estado civil
estado.t <- as.numeric(poli_client$Estado_Civil_cd)
estado2 <- vector()
estado2[estado.t==1] <- 'C'
estado2[estado.t==2] <- 'DS'
estado2[estado.t==3] <- 'DS'# Se agrupan las categorías S y D
estado2[estado.t==4] <- 'U'
estado2[estado.t==5] <- 'V'
estado2 <- as.factor(estado2)
poli_client$estado <- estado2

In [26]:
# Recategorizando la variable ocupación
ocu.t <- as.numeric(poli_client$Ocupacion_Desc)
ocu2 <- vector()
ocu2[ocu.t==175] <- 'I' #Rentistas e independientes
ocu2[ocu.t==197] <- 'I'
ocu2[ocu.t==201] <- 'I'
ocu2[ocu.t==277] <- 'I'
ocu2[ocu.t==288] <- 'sin info'
ocu2[is.na(ocu2)] <- 'E'
ocu2 <- as.factor(ocu2)
poli_client$ocupacion <- ocu2

In [27]:
# Recategorizando la variable estudios
est.t <- as.numeric(poli_client$Nivel_Estudios_Desc)
est2 <- vector()
est2[est.t==1] <- 'O' #bachillerato
est2[est.t==3] <- 'O' #ninguno
est2[est.t==7] <- 'O' #Primaria
est2[est.t==4] <- 'O' #otro
est2[est.t==8] <- 'sin info'  
est2[est.t==9] <- 'T' #técnico
est2[est.t==6] <- 'U' #pregrado
est2[est.t==2] <- 'U' #maestria
est2[est.t==5] <- 'U' #posgrado
est2[est.t==10] <- 'T' #tecno
est2 <- as.factor(est2)
poli_client$estudios <- est2

In [28]:
# Recategorizando la variable departamento
dep.t <- as.numeric(poli_client$Nombre_Departamento)
dep2 <- vector()
dep2[dep.t==3] <- 'ant'
dep2[dep.t==8] <- 'bog'
dep2[dep.t==47] <- 'val'
dep2[dep.t==43] <- 'san'
dep2[dep.t==5] <- 'atl'
dep2[is.na(dep2)] <- 'otro'
dep2 <- as.factor(dep2)
poli_client$depto <- dep2

In [29]:
# Categorizando la variable antiguedad
poli_client$antig <- ifelse(poli_client$antiguedad<1, 1,
                ifelse(poli_client$antiguedad<=2, 2,
                       ifelse(poli_client$antiguedad<=3, 3,
                              ifelse(poli_client$antiguedad<=4, 4,
                                     ifelse(poli_client$antiguedad<=5, 5,
                                            6)))))

Ya con las categorías de las variables organizadas, podemos genrar una base de datos con la información que nos interesa para la estimación de nuestro modelo. 

In [32]:
# Generando la base de datos para la modelación
data <- subset(poli_client, select= c(Cliente_Id, Poliza_Id, cancelada, Sexo_Cd, 
                  Nm_Estrato_Vivienda, Numero_Hijos,edad,
                  ingresos,egresos,estado,ocupacion,estudios,depto,antig)
                  )
head(data)

Cliente_Id,Poliza_Id,cancelada,Sexo_Cd,Nm_Estrato_Vivienda,Numero_Hijos,edad,ingresos,egresos,estado,ocupacion,estudios,depto,antig
<int>,<int>,<dbl>,<fct>,<int>,<int>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>
1,262093074,0,F,NA,NA,26.63014,2,2,C,E,T,ant,4
2,251769194,1,F,3,NA,35.84658,4,2,DS,E,sin info,bog,4
5,261757926,1,F,2,2,38.41370,2,2,DS,I,sin info,val,6
8,255314284,0,M,3,0,29.38630,4,4,DS,E,T,ant,6
22,271897486,1,F,NA,NA,28.54795,4,2,DS,E,sin info,val,4
43,267056618,0,F,NA,0,47.34795,4,4,DS,E,sin info,otro,6


---
## Generar el modelo de regresión logística
---

Se emplea la función $glm()$ con $family="binomial"$ para ajustar modelos de regresión logística. Esta función predice por defecto el $log(ODDs)$ de la variable respuesta.

Para generar este primer modelo se tiene en cuenta todas las variables de las características de los clientes, excepto las de estudios, departamento y ocupación; debido a la gran cantidad de datos faltantes. 

$Nota:$ Se realizaron ejercicios adicionales a los presentados en este documento, donde se determinó que estas variables excluidas no tienen significacia para explicar la probabilidad de cancelación en las categorías agrupadas. 


In [34]:
model1 <- glm(cancelada ~ Sexo_Cd+antig+Nm_Estrato_Vivienda+Numero_Hijos+
               +ingresos+egresos+estado+antig+edad,
              family = binomial, data = data)
summary(model1)


Call:
glm(formula = cancelada ~ Sexo_Cd + antig + Nm_Estrato_Vivienda + 
    Numero_Hijos + +ingresos + egresos + estado + antig + edad, 
    family = binomial, data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7228  -0.7431  -0.6636   0.7701   1.9806  

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)          2.325642   0.098238  23.674  < 2e-16 ***
Sexo_CdM             0.127378   0.025755   4.946 7.58e-07 ***
antig               -0.524775   0.009591 -54.717  < 2e-16 ***
Nm_Estrato_Vivienda -0.063775   0.009493  -6.718 1.84e-11 ***
Numero_Hijos         0.034618   0.008742   3.960 7.50e-05 ***
ingresos1            1.687118   1.382921   1.220 0.222478    
ingresos2            0.024368   0.135985   0.179 0.857782    
ingresos4           -0.016989   0.135796  -0.125 0.900438    
egresos1            -0.801605   1.259271  -0.637 0.524409    
egresos2            -0.414549   0.120052  -3.453 0.000554 ***
egresos3      

---
#### En este caso, el modelo obtenido sí es significativo.

Para determinar si los predictores introducidos en un modelo de regresión logística contribuyen de forma significativa se emplea el estadístico $Z$ y el test Wald chi-test. Este es el método utilizado para calcular los p-values que se muestran al hacer $summary()$ del modelo. 

A diferencia de los modelos de regresión lineal, en los modelos logísticos no existe un equivalente a $R^2$ que determine exactamente la varianza explicada por el modelo. Se han desarrollado diferentes métodos conocidos como $pseudoR^2$ que intentan aproximarse al concepto de $R^2$ pero que, aunque su rango oscila entre 0 y 1, no se pueden considerar equivalentes.

Sin embargo, también se tiene el criterio de $AIC$ (Akaike Information Criteria), el cual es equivalente de $R^2$ en regresión logística. Mide el ajuste cuando se aplica una penalización al número de parámetros. Los valores de AIC más pequeños indican que el modelo está más cerca de la verdad.

Por otra parte, se observa que todas las variables incluídas en el modelo son estadisticamente significativas al 95%, con excepción del nivel de ingresos. 

### Comparación de clasificación predicha y observaciones


Para este caso se va a emplear un threshold de 0.5. Si la probabilidad de que la variable adquiera el valor 1 (póliza cancelada) es superior a 0.5, se asigna a este nivel, si es menor se asigna al 0 (no cancelada).

In [39]:
predict <- predict(model1, data, type = 'response')
table_mat <- table(data$cancelada, predict > 0.5)
table_mat

   
    FALSE  TRUE
  0 23304  1391
  1  7154  3437

In [40]:
accuracy_Test <- sum(diag(table_mat)) / sum(table_mat)
accuracy_Test

[1] 0.757836

El modelo es capáz de clasificar correctamente el 75.78% de las obsercaciones. Si se analiza en detalle cómo se distribuye el error, se aprecia que el modelo solo ha sido capaz de identificar correctamente a 3.437 clientes de 10.591 que cancelan su poliza. El porcentaje de falsos negativos es relativamente alto.

Para el siguiente paso se elimina la variable ingresos de la estimación debido a que no resultó significativa

In [43]:
model2 <- glm(cancelada ~ Sexo_Cd+antig+Nm_Estrato_Vivienda+Numero_Hijos+
               egresos+estado+antig+edad,
              family = binomial, data = data)
summary(model2)


Call:
glm(formula = cancelada ~ Sexo_Cd + antig + Nm_Estrato_Vivienda + 
    Numero_Hijos + egresos + estado + antig + edad, family = binomial, 
    data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7253  -0.7434  -0.6636   0.7696   1.9799  

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)          2.336243   0.093104  25.093  < 2e-16 ***
Sexo_CdM             0.127176   0.025749   4.939 7.85e-07 ***
antig               -0.525841   0.009542 -55.106  < 2e-16 ***
Nm_Estrato_Vivienda -0.064397   0.009471  -6.799 1.05e-11 ***
Numero_Hijos         0.034659   0.008743   3.964 7.36e-05 ***
egresos1             0.117743   0.871947   0.135   0.8926    
egresos2            -0.415337   0.060160  -6.904 5.06e-12 ***
egresos3            -0.542623   0.067462  -8.043 8.74e-16 ***
egresos4            -0.482884   0.060816  -7.940 2.02e-15 ***
estadoDS            -0.131856   0.030167  -4.371 1.24e-05 ***
estadoU              0.259

---
En este caso, las demás variables continuan siendo significativas ademas que el indicador AIC se reduce, aunque marginalmente, indicando que este modelo presenta un mejor ajuste que el aneterior. 

De nuevo, realizamos la comparación de las observaciones frente a la clasificación predicha, usando el mismo umbral que en el caso anterio para probabilidades superiores a 0.5:

In [45]:
predict <- predict(model2, data, type = 'response')
table_mat <- table(data$cancelada, predict > 0.5)
table_mat

   
    FALSE  TRUE
  0 23295  1400
  1  7154  3437

In [46]:
accuracy_Test <- sum(diag(table_mat)) / sum(table_mat)
accuracy_Test

[1] 0.7575809

Observamos que la exactitud del modelo no varía significativamente, en cuanto a que el número de pólizas vigentes que se clasifican como no canceladas y el número de pólizas canceladas que se clasifican como tal es similar a los del modelo anterior.

### Interpretación de los coeficientes 

Ahora bien, una forma de faciliar la interpretación de los coeficientes es exponenciando: 

In [47]:
exp(coefficients(model2))

(Intercept)            Sexo_CdM               antig Nm_Estrato_Vivienda 
         10.3423052           1.1356170           0.5910583           0.9376331 
       Numero_Hijos            egresos1            egresos2            egresos3 
          1.0352670           1.1249544           0.6601181           0.5812217 
           egresos4            estadoDS             estadoU             estadoV 
          0.6170013           0.8764672           1.2959458           0.8023709 
               edad 
          1.0024712

Lo que nos dice esto, por ejemplo, es que un cliente de sexo masculino tiene una mayor probabilidad de cancelar una póliza que un cliente femenino.

La implicaciones de este resultado para la estrategia de rentención y atracción de la empresa aseguradora son de gran importancia, ya que esto nos dice que para la estrategia de retención la compañía se debería centrar en aquellos perfiles de clientes que cumplan con determinadas características. Por ejemplo, un cliente hombre, con una antiguedad menor a 2 años en la empresa (como cliente), con un nivel de ingresos entre 1'200.001 y 3'000.000 y viviendo en unión libre, va a tener mayor probabilidad de cancelar su póliza que una mujer casada, con un mayor nivel de ingresos y sin hijos. 